In [ ]:
%%writefile create_veterans_parallel.py

import math
import ipyparallel
import itertools

# at terminal : ipcluster start -n 4

clients = ipyparallel.Client()
dview = clients.direct_view()

with dview.sync_imports():
    from mouseworld import mouseworld
    from mouseworld.mouseworld import Mouseworld
    import time
dview.push({"Mouseworld": Mouseworld})

def run_experiment() :
#    
    num_mice = [0, 0, 100]

    # Build the model
    print('Building mouseworld')
    model = mouseworld.Mouseworld(num_mice, 100, 0, mouse_initial_energy = 1000, mouse_max_energy = 1200,
                     food_amount_range = (20,40), nutritional_value = [1], mouse_reproduction = False, brain_iterations_per_step = 10)

    # Gather initial randomly distributed data
    # model.initial_datacollector.collect(model,model.schedule)
    # initial_model_data = model.initial_datacollector.get_model_vars_dataframe()
    # initial_model_data.to_csv('results/initial_model_data.csv', sep='\t')

    # Prepare environment by stepping food and predators and diffusing odors
    print('Preparing environment')
    for i in range(40) :
        model.food_schedule.step()
        model.predator_schedule.step()
        model.diffuse_odor_layers(model.odor_layers)

    counter = 0
    myrange = 60
    # Run for discrete number of timesteps
#     print('Simulatimg for %i timesteps'%myrange)
#     for i in range(myrange) :
#         c=time.time()
#         counter += 1
#         model.step()
#         d=time.time()
#         print('sim step : %i in %f'%(counter, d-c))
#     print('Simulation terminated - Number of time steps reached')

    # Run until all mice perish
    print('Simulatimg until all mice perish')
    while model.num_mice > 0 :
        c=time.time()
        counter += 1
        model.step()
        d=time.time()
        print('sim step : %i in %f'%(counter, d-c))
    print('Simulation terminated - No alive mice')
        
    # Gather final model and agent data
    model.final_datacollector.collect(model,model.all_mice_schedule)
    # final_model_data = model.final_datacollector.get_model_vars_dataframe()
    # print(final_model_data)

    final_agent_data = model.final_datacollector.get_agent_vars_dataframe()
    mouse_statistics = final_agent_data[['first_action_duration', 'first_action_termination']]
    # mouse_statistics = final_agent_data['action_history']
    # mouse_statistics = final_agent_data['secondary_values']
    # mouse_statistics = final_agent_data['sensor_vector']

    mouse_statistics = mouse_statistics.reset_index('Step', drop = True)
        #mouse_statistics = mouse_statistics.reset_index('AgentID', drop = True)
    num_trials = len(mouse_statistics.index)
    # for i in range(num_trials)
    succesful_trials = mouse_statistics.loc[(mouse_statistics['first_action_termination'] == 'Closure')]
    unsuccesful_trials = mouse_statistics.loc[(mouse_statistics['first_action_termination'] == 'Failure')]
    # incomplete_trials = mouse_statistics.loc[(mouse_statistics['first_action_termination'] == 'None')]

    num_succesful_trials = len(succesful_trials.index)
    num_unsuccesful_trials = len(unsuccesful_trials.index)
    # num_incomplete_trials = len(incomplete_trials.index)

    #performance = num_succesful_trials / num_trials
    mean_time_success = succesful_trials['first_action_duration'].mean()
    mean_time_failure = unsuccesful_trials['first_action_duration'].mean()

    sim_params = [mouse_position, num_mice, myrange]
    results = [num_succesful_trials, mean_time_success, num_unsuccesful_trials, mean_time_failure]
    exp_data = [results, sim_params]
    return exp_data
    # mouse_statistics = [performance,  mean_time]
    #     sensor_vector = final_agent_data['sensor_vector'][0].values[0]
#     sensor_position = final_agent_data['sensor_position'][0].values[0]
#     motor_vector = final_agent_data['motor_vector'][0].values[0]
#     first_action = final_agent_data['action_history'][0].values[0].loc[0]
#     first_action = mousetest_data
#     first_action = (mousetest_data['Duration'], mousetest_data['Termination'])
    #first_action = final_agent_data['action_history'][0].values[0].loc[0]
#     return (performance,  mean_time)
#     return (mouse_statistics)
# print(performance,  mean_time)
all_exp_data = dview.map_sync(run_experiment, mouse_positions)

file = open('results/check_approach_consistency_001_2.txt','w') 
for exp_data in all_exp_data :
    file.write(str(exp_data) + '\n')
# file.write(str(results) + '\n')
file.close() 
# mouse_statistics.to_csv('results/mouse_statistics.csv', sep='\t')

In [1]:
#%%writefile create_veterans.py

from mouseworld import mouseworld
import time

num_mice = [0, 0, 10]

 # Build the model
print('Building mouseworld')
model = mouseworld.Mouseworld(num_mice, 100, 0, mouse_initial_energy = 1000, mouse_max_energy = 1200,
                 food_amount_range = (20,40), nutritional_value = [1], mouse_reproduction = False, 
                              brain_iterations_per_step = 10, mousebrain_seed = 5)

# Gather initial randomly distributed data
# model.initial_datacollector.collect(model,model.schedule)
# initial_model_data = model.initial_datacollector.get_model_vars_dataframe()
# initial_model_data.to_csv('results/initial_model_data.csv', sep='\t')

# Prepare environment by stepping food and predators and diffusing odors
print('Preparing environment')
for i in range(40) :
    model.food_schedule.step()
    model.predator_schedule.step()
    model.diffuse_odor_layers(model.odor_layers)

counter = 0
myrange = 2
# Run for discrete number of timesteps
# print('Simulatimg for %i timesteps'%myrange)
# for i in range(myrange) :
#     c=time.time()
#     counter += 1
#     model.step()
#     d=time.time()
#     print('sim step : %i in %f'%(counter, d-c))
# print('Simulation terminated - Number of time steps reached')


# Run until all mice perish
print('Simulatimg until all mice perish')
while model.num_mice > 0 :
    c=time.time()
    counter += 1
    model.step()
    d=time.time()
    print('sim step : %i in %f'%(counter, d-c))
print('Simulation terminated - No alive mice')

print('Storing mousebrains')
# print(model.exp_approach_rank)
# print(model.exp_approach_rank[:1])
for i in model.exp_approach_rank[:5] :
    i.store_mousebrain_weights()
    
# # Gather final model and agent data
# model.final_datacollector.collect(model,model.all_mice_schedule)
# # final_model_data = model.final_datacollector.get_model_vars_dataframe()
# # print(final_model_data)

# final_agent_data = model.final_datacollector.get_agent_vars_dataframe()
# mouse_statistics = final_agent_data[['first_action_duration', 'first_action_termination']]


Building mouseworld


/home/bagjohn/anaconda3/lib/python3.6/site-packages/nengo/node.py:28: UserWarning: 'Node.size_out' is being overwritten with 'Node.size_in' since 'Node.output=None'
  warnings.warn("'Node.size_out' is being overwritten with "


Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Building finished in 0:00:01.                                                   
Preparing environment
Simulatimg until all mice perish


/home/bagjohn/Documents/PhD/mouseworld_python/mouseworld/mouse.py:272: RuntimeWarning: invalid value encountered in double_scalars
  sensor_vector[i] = [np.mean(sensor_vector[i]), (sensor_vector[i][0]-sensor_vector[i][1])/np.mean(sensor_vector[i])]
/home/bagjohn/Documents/PhD/mouseworld_python/mouseworld/mouse.py:586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.action_history['Distance'][self.action_history.index.max()] += distance
/home/bagjohn/Documents/PhD/mouseworld_python/mouseworld/mouse.py:538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.action_history['Benefit'][self.action_history.index.max()] += gain


sim step : 1 in 1.427053


...

sim step : 3 in 2.115228
sim step : 4 in 2.063942
sim step : 5 in 2.107656
sim step : 6 in 1.981898
sim step : 7 in 2.145622
sim step : 8 in 2.259495
sim step : 9 in 2.179507
sim step : 10 in 2.005031
sim step : 11 in 2.019527
sim step : 12 in 2.142473
sim step : 13 in 2.190406
sim step : 14 in 2.388853
sim step : 15 in 2.146938
sim step : 16 in 2.349213
sim step : 17 in 1.972371
sim step : 18 in 2.135364
sim step : 19 in 2.556731
sim step : 20 in 2.221015


/home/bagjohn/Documents/PhD/mouseworld_python/mouseworld/mouse.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.action_history['Termination'][self.action_history.index.max()] = 'Death'


sim step : 21 in 2.252279
sim step : 22 in 1.813278
sim step : 23 in 2.136048
sim step : 24 in 2.103550


/home/bagjohn/Documents/PhD/mouseworld_python/mouseworld/mouse.py:325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.action_history['Termination'][self.action_history.index.max()] = 'Closure'


sim step : 25 in 2.373126
sim step : 26 in 1.948687
sim step : 27 in 1.670238
sim step : 28 in 1.686309
sim step : 29 in 1.950197
sim step : 30 in 1.844458
sim step : 31 in 2.278769
sim step : 32 in 1.659572
sim step : 33 in 1.818720
sim step : 34 in 2.104428
sim step : 35 in 1.914198
sim step : 36 in 2.254445
sim step : 37 in 1.513936
sim step : 38 in 1.563665
sim step : 39 in 1.610671
sim step : 40 in 1.402482
sim step : 41 in 1.404395
sim step : 42 in 1.384251
sim step : 43 in 1.238401
sim step : 44 in 1.190616
sim step : 45 in 1.219003
sim step : 46 in 1.160505
sim step : 47 in 1.209266
sim step : 48 in 1.484375
sim step : 49 in 1.129452
sim step : 50 in 1.218439
sim step : 51 in 1.145310
sim step : 52 in 1.211511
sim step : 53 in 1.189660
sim step : 54 in 1.187799
sim step : 55 in 1.185236
sim step : 56 in 1.243910
sim step : 57 in 1.135929
sim step : 58 in 1.241760
sim step : 59 in 1.162730
sim step : 60 in 1.105846
sim step : 61 in 1.122503
sim step : 62 in 1.112264
sim step : 6

In [11]:
%%writefile test_veterans.py

from mouseworld import mouseworld
import time
import sys
import numpy as np

mouse_data = np.load(sys.argv[1])

if mouse_data['motor_NN_on'] :
    if mouse_data['learning_on'] :
        num_mice = [0, 0, 152]
    else :
        num_mice = [0, 152, 0]
else :
    num_mice = [152, 0, 0]
genome = np.around(mouse_data['genome'], decimals = 2)
genome_range = [(x,x) for x in genome]
mousebrain_seed = mouse_data['seed']
brain_iterations_per_step = mouse_data['brain_iterations_per_step']
mousebrain_steps = mouse_data['mousebrain_steps']
mousebrain_weights = [mouse_data['w_search'], mouse_data['w_approach'], mouse_data['w_avoid']]
                 
# Build the model
print('Building mouseworld')
model = mouseworld.Mouseworld(num_mice, 1, 0, genome_range = genome_range, 
                              mouse_position = 'in_quadrant', food_position = (0,0), 
                              primary_values = [100, 0], secondary_values = [100,0], food_amount_range = (10000,10000), 
                              nutritional_value = [1], mouse_reproduction = False, 
                              brain_iterations_per_step = brain_iterations_per_step, mousebrain_seed = mousebrain_seed,
                             initial_mousebrain_weights = mousebrain_weights, test_veteran = True)

# Gather initial randomly distributed data
# model.initial_datacollector.collect(model,model.schedule)
# initial_model_data = model.initial_datacollector.get_model_vars_dataframe()
# initial_model_data.to_csv('results/initial_model_data.csv', sep='\t')

# Prepare environment by stepping food and predators and diffusing odors
print('Preparing environment')
for i in range(40) :
    model.food_schedule.step()
    model.predator_schedule.step()
    model.diffuse_odor_layers(model.odor_layers)

counter = 0
myrange = 40
# Run for discrete number of timesteps
print('Simulatimg for %i timesteps'%myrange)
for i in range(myrange) :
    c=time.time()
    counter += 1
    model.step()
    d=time.time()
    print('sim step : %i in %f'%(counter, d-c))
print('Simulation terminated - Number of time steps reached')


# Run until all mice perish
# print('Simulatimg until all mice perish')
# while model.num_mice > 0 :
#     c=time.time()
#     counter += 1
#     model.step()
#     d=time.time()
#     print('sim step : %i in %f'%(counter, d-c))
# print('Simulation terminated - No alive mice')
    
# Gather final model and agent data
print('Gathering simulation data')
model.final_datacollector.collect(model,model.all_mice_schedule)

final_agent_data = model.final_datacollector.get_agent_vars_dataframe()
mouse_statistics = final_agent_data[['control_population', 'first_action_duration', 'first_action_termination']]

mouse_statistics = mouse_statistics.reset_index('Step', drop = True)
print('Counting successful trials')
veteran_stats = mouse_statistics.loc[mouse_statistics['control_population'] == False]
control_stats = mouse_statistics.loc[mouse_statistics['control_population'] == True]
num_trials = len(veteran_stats.index)
succesful_veteran_trials = veteran_stats.loc[(veteran_stats['first_action_termination'] == 'Closure')]
succesful_control_trials = control_stats.loc[(control_stats['first_action_termination'] == 'Closure')]
unsuccesful_veteran_trials = veteran_stats.loc[(veteran_stats['first_action_termination'] == 'Failure')]
unsuccesful_control_trials = control_stats.loc[(control_stats['first_action_termination'] == 'Failure')]

num_succesful_veteran_trials = len(succesful_veteran_trials.index)
num_succesful_control_trials = len(succesful_control_trials.index)                                     
num_unsuccesful_veteran_trials = len(unsuccesful_veteran_trials.index)
num_unsuccesful_control_trials = len(unsuccesful_control_trials.index)

mean_time_veteran_success = succesful_veteran_trials['first_action_duration'].mean()
mean_time_control_success = succesful_control_trials['first_action_duration'].mean()                 
mean_time_veteran_failure = unsuccesful_veteran_trials['first_action_duration'].mean()
mean_time_control_failure = unsuccesful_control_trials['first_action_duration'].mean()

veteran_results = [num_succesful_veteran_trials, mean_time_veteran_success, num_unsuccesful_veteran_trials, mean_time_veteran_failure]
control_results = [num_succesful_control_trials, mean_time_control_success, num_unsuccesful_control_trials, mean_time_control_failure]

file = open('results/veteran_mousebrains/test_veterans.txt','w') 
file.write('experience : %s'%str(mousebrain_steps) + '\t')
file.write('mousebrain seed : %s' %str(mousebrain_seed) + '\n')
file.write('veteran results' + '\t')
file.write(str(veteran_results) + '\n')
file.write('control results' + '\t')
file.write(str(control_results) + '\n')

file.close() 


Overwriting test_veterans.py
